In [1]:
# Import MySQL Connector/Python 
import mysql.connector as connector

connection=connector.connect(user="root",password="51125112")

In [2]:
cursor = connection.cursor()

In [3]:
cursor.execute("USE little_lemon_db")

In [4]:
#################################################
#                Task 1                         #
#################################################
print("First task:")

First task:


In [6]:
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error

dbconfig = {
    "database":"little_lemon_db",
    "user":"root",
    "password":"51125112"
}
try:
    pool_b = MySQLConnectionPool(pool_name="pool_b",
                               pool_size=2,
                               **dbconfig)
    print("The conncetion pool is created with the name: ", pool_b.pool_name)
    print("The pool size is: ", pool_b.pool_size)
except Error as er:
    print("Error code: ", er.errno)
    

The conncetion pool is created with the name:  pool_b
The pool size is:  2


In [7]:
#################################################
#                Task 2                         #
#################################################
print("Second task:")

Second task:


In [8]:
try:
    # Get connections from the pool
    connection1 = pool_b.get_connection()
    connection2 = pool_b.get_connection()
    connection3 = pool_b.get_connection()
    
    cursor1 = connection1.cursor()
    cursor1.execute("""
        INSERT INTO Bookings (TableNumber, FirstName, LastName, BookingSlot, EmployeeID)
        VALUES (8, 'Anees', 'Java', '18:00', 6)
    """)
    connection1.commit()

    
    cursor2 = connection2.cursor()
    cursor2.execute("""
        INSERT INTO Bookings (TableNumber, FirstName, LastName, BookingSlot, EmployeeID)
        VALUES (5, 'Bald', 'Vin', '19:00', 6)
    """)
    connection2.commit()

    
    cursor3 = connection3.cursor()
    cursor3.execute("""
        INSERT INTO Bookings (TableNumber, FirstName, LastName, BookingSlot, EmployeeID)
        VALUES (12, 'Jay', 'Kon', '19:30', 6)
    """)
    connection3.commit()

except Error as er:
    print("Error code:", er.errno)
    print("Error message:", er.msg)



Error code: -1
Error message: Failed getting connection; pool exhausted


In [9]:
#################################################
#                Task 3                         #
#################################################
print("Third task:")

Third task:


In [10]:
try:
    manager = """SELECT Name, EmployeeID FROM Employees WHERE Role = 'Manager' """
    hig_sal = """SELECT Name, Role FROM Employees ORDER BY Annual_Salary DESC LIMIT 1"""
    n_guest = """SELECT COUNT(*) FROM Bookings WHERE HOUR(BookingSlot) BETWEEN 18 AND 20"""
    seat = """ SELECT CONCAT(GuestFirstName, '', GuestLastName) AS GuestName, BookingID FROM Bookings ORDER BY HOUR(BookingSlot) ASC"""

    cursor.execute(manager)
    manager_info = cursor.fetchone()

    cursor.execute(hig_sal)
    hig_sal_info = cursor.fetchone()

    cursor.execute(n_guest)
    n_guest_info = cursor.fetchone()

    cursor.execute(seat)
    seat_info = cursor.fetchall()

    report = {
        "Manager": manager_info,
        "Highest Salary": hig_sal_info,
        "Guests Count": n_guest_info,
        "Waiting Guests": seat_info
        }

    print("Report generated successfully")
    print(report)
except Error as er:
    print(f"Error: {er}")
    

Report generated successfully
{'Manager': ('Mario Gollini', 1), 'Highest Salary': ('Mario Gollini', 'Manager'), 'Guests Count': (4,), 'Waiting Guests': [('VanessaMcCarthy', 3), ('MarcosRomero', 4), ('HirokiYamane', 5), ('AnnaIversen', 1), ('JoakimIversen', 2), ('DianaPinto', 6)]}


In [11]:
#################################################
#                Task 4                         #
#################################################
print("Fourth task:")

Fourth task:


In [12]:
cursor.execute("DROP PROCEDURE IF EXISTS BasicSalesReport;")

proc = """
CREATE PROCEDURE BasicSalesReport()
BEGIN
    SELECT
        SUM(BillAmount) AS TotalSales,
        AVG(BillAmount) AS AverageSale,
        MIN(BillAmount) AS MinimumBill,
        MAX(BillAmount) AS MaximumBill
    FROM Orders;
END
"""

cursor.execute(proc)
cursor.callproc("BasicSalesReport")

results = next(cursor.stored_results())
dataset = results.fetchall()

columns = [column[0] for column in results.description]
print(columns)

for data in dataset:
    print(data)


['TotalSales', 'AverageSale', 'MinimumBill', 'MaximumBill']
(Decimal('243'), Decimal('48.6000'), 37, 86)


In [13]:
#################################################
#                Task 5                         #
#################################################
print("Fifth task:")

Fifth task:


In [14]:
try:
    pool_b = MySQLConnectionPool(pool_name="pool_b", pool_size=2, **dbconfig)
    print("Connection pool created successfully")
except Error as e:
    print(f"Error: {e}")


try:
        
        connection = pool_b.get_connection()
        cursor = connection.cursor(buffered=True)
        
        query = """
        SELECT 
            Bookings.BookingSlot AS BookingSlot, 
            CONCAT(Bookings.GuestFirstName, ' ', Bookings.GuestLastName) AS Guest_name, 
            CONCAT('Assigned to: ', Employees.Name, ' [', Employees.Role, ']') AS Assigned_to
        FROM Bookings
        JOIN Employees ON Bookings.EmployeeID = Employees.EmployeeID
        ORDER BY BookingSlot ASC
        LIMIT 3;
        """
        
        cursor.execute(query)
        results = cursor.fetchall()
        
        for row in results:
            print(f"[{row[0]}]\n[{row[1]}]\n[{row[2]}]\n")
        
        # Return the connection back to the pool
        cursor.close()
        connection.close()
        
except Error as e:
        print(f"Error: {e}")

Connection pool created successfully
[15:00:00]
[Vanessa McCarthy]
[Assigned to: Giorgos Dioudis [Head Chef]]

[17:30:00]
[Marcos Romero]
[Assigned to: Fatma Kaya [Assistant Chef]]

[18:30:00]
[Hiroki Yamane]
[Assigned to: Adrian Gollini [Assistant Manager]]



In [15]:
#Closing the Connection 

if connection.is_connected():
    cursor.close()
    print("Cursor is Closed")
    connection.close()
    print("Connection is Closed")
else:
    print("Connection is already closed")

AttributeError: 'NoneType' object has no attribute 'is_connected'